In [ ]:
from fastai.vision import *

import pose
import models.cs_v2 as cs

#%
root = Path('../../LIP').resolve()

In [ ]:
class RecurrentInstructor(cs.BaseInstructor):
    def __init__(self, repeats):
        self.repeats = repeats
        
    def on_batch_begin(self, **kwargs):
        self.i = 0
    
    def next_inst(self, last_bu):
        self.i += 1
        state = {'continue': self.i < self.repeats}
        return None, state
    
class RecurrentLoss:
    
    def __init__(self, repeats):
        self.r = repeats
        
    def __call__(self, outputs, targets):
        targets = targets.repeat(1, self.r, 1)
        return pose.pose_ce_loss(outputs[1], targets)
    
       
pckh = partial(pose.Pckh, heatmap_func=lambda last_output: last_output[1][:, -16:])

In [ ]:
data = pose.get_data(root, 128)
data

In [ ]:
r = 4
instructor = RecurrentInstructor(r)
learner = cs.cs_learner(data, models.resnet18, instructor, td_c=16, pretrained=False, embedding=None,
                        loss_func=RecurrentLoss(r), callback_fns=pckh)

In [ ]:
learner.lr_find()
learner.recorder.plot(suggestion=True)

In [ ]:
learner.fit_one_cycle(40, 1e-2)

In [ ]:
learner.save(f'repeat{r}-128')
